In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

In [ ]:
# Data from sde_simulation.ipynb
df_res = pd.read_csv('./train_data/train_res.csv')
df_sm = pd.read_csv('./train_data/train_sm.csv')
df_lb = pd.read_csv('./train_data/train_lb.csv')

In [ ]:
# Shuffle the training set
df_res = df_res.sample(frac=1, axis=1, random_state=999)
df_sm = df_sm.sample(frac=1, axis=1, random_state=999)
df_lb = df_lb.sample(frac=1, random_state=999).reset_index(drop=True)

In [ ]:
n_steps_in, n_steps_out, n_features = 500, 1, 2

In [ ]:
def reshape_inputs(df1, df2, n_features=2):
    # Ensure df1 and df2 are in the correct shape and convert them to NumPy arrays
    arr1 = df1.values.T.reshape(-1, df1.shape[0], 1)  # Shape: (n_samples, n_time_steps, 1)
    arr2 = df2.values.T.reshape(-1, df2.shape[0], 1)  # Shape: (n_samples, n_time_steps, 1)
    
    # Stack the arrays along the last dimension to combine the features
    X = np.concatenate((arr1, arr2), axis=2)  # Shape: (n_samples, n_time_steps, n_features)
    
    return X

In [ ]:
def residual_block(x, filters=50):
    # First convolution layer
    h = tf.keras.layers.Conv1D(filters=filters, kernel_size=8, padding='same', kernel_initializer='lecun_normal')(x)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)
    h = tf.keras.layers.Dropout(rate=0.1)(h)

    # Second convolution layer
    h = tf.keras.layers.Conv1D(filters=filters, kernel_size=5, padding='same', kernel_initializer='lecun_normal')(h)
    h = tf.keras.layers.BatchNormalization()(h)
    h = tf.keras.layers.Activation('relu')(h)
    h = tf.keras.layers.Dropout(rate=0.1)(h)

    # Third convolution layer
    h = tf.keras.layers.Conv1D(filters=filters, kernel_size=3, padding='same', kernel_initializer='lecun_normal')(h)
    h = tf.keras.layers.BatchNormalization()(h)
    
    # Add the shortcut (identity) to the output
    x = tf.keras.layers.BatchNormalization()(x)
    y = tf.keras.layers.add([x, h])
    y = tf.keras.layers.Activation('tanh')(y)

    return y

In [ ]:
inputs = reshape_inputs(df_res, df_sm)

In [ ]:
inputs.shape

In [ ]:
targets = np.array(df_lb.values.tolist())

In [ ]:
targets.shape

In [ ]:
input_shape = (500,2)
learning_rate_param = 0.0005
batch_param = 1024
epoch_param = 300
initializer_param = 'lecun_normal'

In [ ]:
# Model
input_layer = tf.keras.layers.Input(input_shape)
x1 = tf.keras.layers.Conv1D(filters=50, kernel_size=12, padding='same', kernel_initializer=initializer_param)(input_layer)
y1 = residual_block(x1, filters=50)
x2 = tf.keras.layers.Conv1D(filters=100, kernel_size=12, padding='same', kernel_initializer=initializer_param)(y1)
y2 = residual_block(x2, filters=100)
x3 = tf.keras.layers.Conv1D(filters=100, kernel_size=12, padding='same', kernel_initializer=initializer_param)(y2)
y3 = residual_block(x3, filters=100)
y = tf.keras.layers.GlobalAveragePooling1D()(y3)
output_layer = tf.keras.layers.Dense(n_steps_out, activation='sigmoid', kernel_initializer=initializer_param)(y)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate_param)
model.compile(optimizer=opt, loss='binary_crossentropy')

In [ ]:
model.summary()

In [ ]:
# Fit model
model.fit(inputs, targets, epochs=epoch_param, batch_size=batch_param, verbose=2)

In [ ]:
model.save('./trained_models/resnet_300ep_1024batch.h5')